In [12]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [34]:
url = 'https://www.nfa.go.kr/nfa/introduce/status/firestationidfo/?searchDistance=10&searchMode=distance&myX=37.5129582&myY=126.9193489&searchKeyword='
driver = webdriver.Chrome()
driver.get(url)

In [14]:
# 검색 입력창에 서울 입력하기
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')

In [15]:
# 검색 버튼 클릭하기
driver.find_element(By.ID, 'fsSearchBtn').click()

In [16]:
# 페이지 이동해서 HTML 코드를 BeautifulSoup으로 parsing
soup = BeautifulSoup(driver.page_source, 'html.parser')
lis = soup.select('.stations-list > li')
len(lis)

10

In [17]:
# 기관명, 주소, 전화번호
li = lis[0]
title = li.select_one('.title').get_text().strip()
addr = li.find('address').get_text().strip()
tel = li.select_one('.tel').get_text().strip()
title, addr, tel

('동작소방서', '서울특별시 동작구 여의대방로16길 55(신대방동)', '02-847-1190')

In [18]:
data = []
for li in lis:
    title = li.select_one('.title').get_text().strip()
    addr = li.find('address').get_text().strip()
    tel = li.select_one('.tel').get_text().strip()
    data.append({'기관명': title, '주소': addr, '전화번호': tel})

In [19]:
df = pd.DataFrame(data)
df.head()

,기관명,주소,전화번호
0,동작소방서,서울특별시 동작구 여의대방로16길 55(신대방동),02-847-1190
1,서대문소방서,서울특별시 서대문구 연희로 182(연희동),02-3144-1190
2,광진소방서,서울특별시 광진구 광나루로 480(구의동),02-457-0119
3,송파소방서,서울특별시 송파구 오금로51길 56(마천동),02-403-2119
4,양천소방서,서울특별시 양천구 목동서로 180(목동),02-2655-1119


- 두번째 페이지로 이동

In [20]:
# //*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a       우클릭 > copy > copy XPATH
driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()

- next(>) 페이지로 이동

In [21]:
driver.find_element(By.CSS_SELECTOR, '.next_page').click()
# driver.find_element(By.CLASS_NAME, 'next_page')

- 모든 페이지에 대해서 적용

In [35]:
# 첫번째 페이지로 이동
driver.get(url)
time.sleep(1)
driver.find_element(By.ID, 'searchKeyword').send_keys('서울')
driver.find_element(By.ID, 'fsSearchBtn').click()

In [36]:
# 검색결과 건수 확인
count = driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/div/p/strong[2]').text
count = int(count[:-1])
count

100

In [37]:
import math
total_pages = math.ceil(count/10)
total_pages

10

In [38]:
from tqdm import tqdm

In [39]:
data = []
for i in tqdm(range(total_pages)):
    if i % 2 == 1:
        driver.find_element(By.XPATH, '//*[@id="listForm"]/div/section/ul/li[1]/div/div/ul/li[4]/a').click()
        time.sleep(1)
    if i > 0 and i % 2 == 0:
        driver.find_element(By.CSS_SELECTOR, '.next_page').click()
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('.stations-list > li')

    for li in lis:
        title = li.select_one('.title').get_text().strip()
        addr = li.find('address').get_text().strip()
        tel = li.select_one('.tel').get_text().strip()
        data.append({'기관명': title, '주소': addr, '전화번호': tel})
        
driver.close()

100%|██████████| 10/10 [00:11<00:00,  1.10s/it]


In [40]:
df = pd.DataFrame(data)
df.tail(20)

,기관명,주소,전화번호
80,종로소방서-종로-119 안전센터,서울특별시 종로구 돈화문로 30 (묘동),02-764-0119
81,종로소방서-신교-119 안전센터,서울특별시 종로구 자하문로 93 (신교동),02-736-0119
82,종로소방서-세종로-119 안전센터,서울특별시 종로구 종로1길 28 (수송동),02-734-0119
83,강북소방서_서울-삼각산-119 안전센터,서울특별시 강북구 솔샘로 208 (미아동),02-983-5119
84,강북소방서_서울-미아-119 안전센터,서울특별시 강북구 도봉로 125 (미아동),02-983-9111
85,강북소방서_서울-우이-119 안전센터,서울특별시 강북구 삼각산로 111 (수유동),02-998-9116
86,구로소방서-구로_공단-119 안전센터,서울특별시 구로구 디지털로34길 56 (구로동),02-855-0119
87,구로소방서-신도림-119 안전센터,서울특별시 구로구 경인로59길 57 (신도림동),02-2671-8119
88,구로소방서-구로_시흥-119 안전센터,서울특별시 금천구 시흥대로45길 31 (시흥동),02-806-1119
89,구로소방서-독산-119 안전센터,서울특별시 금천구 시흥대로 386 (독산동),02-869-1119
